# Interviewer Occupation Detection 

Authors: Ahmed Z. , Cohen H. and Spence S.

# Introduction
(overview of approach)


# Related Work





# Data Pre-Processing Methods

Hannah maybe you can fill this part up?

# feature design / selection methods

have we designed any features? 
How about is_interviewer or is_interviewee



# Algorithm Selection

## Baseline

Picked a Bernoulli Naive Bayes and made it multiclass using the 1 vs all approach. We also used indicator bag of words rather than a frequency bag of words, this is because the exerpts were short enough that the words will not appear often enough to be counted highly 
(i have analysis of this.... will put that in here....)

Maybe I can also find words that appear with frequency 2 multiple times and add that in as another binary feature?

## Standard

Stuarts speil on decision trees?

## Advanced

We need to get on with this ASAP

# Optimization 

- if any used

# Parameter selection

- here I can probably do an analysis of which gamma works best.

# Testing + Validation 

- done outside of Kaggle

# Pros and Cons of approach and methodology

We	hereby	state	that	all	the	work presented	in this	report	is	that	of	the	authors

Statement of Contributions

- Zafarali Ahmed
- Stuart Spence
- Hannah Cohen